In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/Medical Projects

/content/drive/.shortcut-targets-by-id/1NQnAFqHSXiU93mL4Dj8JBo3fbVPBUujA/Medical Projects


## Init

In [ ]:
import logging
import os
logger = logging.getLogger()
logger = logging.getLogger('my-logger')
logger.propagate = False

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

## Import libraries

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.models import model_from_json
import json
import cv2
import numpy as np
from skimage.io import imsave

import numpy as np
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import img_to_array, load_img
from PIL import Image
from numpy import savez_compressed
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from keras import backend as K
from sklearn.model_selection import train_test_split

from numpy import savez_compressed
from numpy import load
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model, load_model
import matplotlib.pyplot as plt
from random import randint
from keras.metrics import CategoricalAccuracy, Recall, Precision
import pickle

## Prepare data

In [ ]:
root_path = "right_coronary_artery/dataset/startp_detection/"

In [ ]:
# load all images in a directory into memory
def load_images(size=(512,512)):
  X_data_list = list()
  y_data_list = list()
  count = 0

  data_path = root_path

	# enumerate filenames in directory, assume all are images
  for filename in os.listdir(data_path + 'X/'):
    X_path = data_path + 'X/' + filename
    y_path = data_path + 'y/' + filename

		# load and resize the image
    X_pixels = load_img(X_path, target_size=size, grayscale=True)

		# convert to numpy array: (512, 512, 1)
    X_pixels = img_to_array(X_pixels)

    # scale from [0,255] to [0,1]
    X_pixels = X_pixels/255

    # store
    X_data_list.append(X_pixels)
    print(count)
    count += 1

    # load and resize the image
    y_pixels = load_img(y_path, target_size=size, grayscale=True)

    # convert to numpy array: (512, 512, 1)
    y_pixels = img_to_array(y_pixels)

    # scale from [0,255] to [0,1]
    y_pixels = y_pixels/255

    # store

    y_data_list.append(y_pixels)
  return np.asarray(X_data_list), np.asarray(y_data_list)

# load dataset A
X, y = load_images()

/usr/local/lib/python3.10/dist-packages/keras/src/utils/image_utils.py:409: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn(


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
# split data
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=44,
)

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))

## Attention

In [ ]:
def iou_score(y_pred, y_true, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    union = K.sum(y_true, -1) + K.sum(y_pred, -1) - intersection
    iou = (intersection + smooth)/(union + smooth)
    return iou

def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

In [ ]:
def AttnBlock2D(x, g, inter_channel, data_format='channels_first'):

    theta_x = Conv2D(inter_channel, [1, 1], strides=[1, 1], data_format=data_format)(x)

    phi_g = Conv2D(inter_channel, [1, 1], strides=[1, 1], data_format=data_format)(g)

    f = Activation('relu')(add([theta_x, phi_g]))

    psi_f = Conv2D(1, [1, 1], strides=[1, 1], data_format=data_format)(f)

    rate = Activation('sigmoid')(psi_f)

    att_x = multiply([x, rate])

    return att_x


def attention_up_and_concate(down_layer, layer, data_format='channels_first'):

    if data_format == 'channels_first':
        in_channel = down_layer.get_shape().as_list()[1]
    else:
        in_channel = down_layer.get_shape().as_list()[3]

    up = UpSampling2D(size=(2, 2), data_format=data_format)(down_layer)
    layer = AttnBlock2D(x=layer, g=up, inter_channel=in_channel // 4, data_format=data_format)

    if data_format == 'channels_first':
        my_concat = Lambda(lambda x: K.concatenate([x[0], x[1]], axis=1))
    else:
        my_concat = Lambda(lambda x: K.concatenate([x[0], x[3]], axis=3))

    concate = my_concat([up, layer])
    return concate

In [ ]:
# Attention U-Net
def att_unet(img_w, img_h, n_label, data_format='channels_first'):
    inputs = Input((1, img_w, img_h))
    x = inputs
    depth = 4
    features = 32
    skips = []
    for i in range(depth):

        # ENCODER
        x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)
        x = Dropout(0.2)(x)
        x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)
        skips.append(x)
        x = MaxPooling2D((2, 2), data_format='channels_first')(x)
        features = features * 2

    # BOTTLENECK
    x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)
    x = Dropout(0.2)(x)
    x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)

    # DECODER
    for i in reversed(range(depth)):
        features = features // 2
        x = attention_up_and_concate(x, skips[i], data_format=data_format)
        x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)
        x = Dropout(0.2)(x)
        x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)

    conv6 = Conv2D(n_label, (1, 1), padding='same', data_format=data_format)(x)
    conv7 = Activation('sigmoid')(conv6)

    model = Model(inputs=inputs, outputs=conv7)

    return model

In [ ]:
model = att_unet(512, 512, n_label=1)

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[dice_coef_loss, iou_score]
)

## Data Augmentation

In [ ]:
from tensorflow.keras import layers
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal_and_vertical",
                      input_shape=(512, 512, 1)),
    # layers.RandomRotation(factor=0.4, fill_mode="wrap"),
    # layers.RandomContrast(factor=0.2),
    # layers.RandomZoom(height_factor=0.4, width_factor=0.4),
  ]
)

In [ ]:
def fix_x(x):
  return np.rollaxis(x, 2, 0)

In [ ]:
tf.experimental.numpy.experimental_enable_numpy_behavior()

In [ ]:
BATCH_SIZE = 2**3
AUTOTUNE = tf.data.AUTOTUNE
def prepare(ds, shuffle=False, augment=False):
    ds = ds.map(lambda x, y: (fix_x(x), fix_x(y)),
              num_parallel_calls=AUTOTUNE)

    if shuffle:
        ds = ds.shuffle(1000)

    # Batch all datasets.
    ds = ds.batch(BATCH_SIZE)

    # Use data augmentation only on the training set.
    if augment:
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y),
                num_parallel_calls=AUTOTUNE)

    # Use buffered prefetching on all datasets.
    return ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
train_ds = prepare(train_ds, shuffle=True, augment=False)
# val_ds = prepare(val_ds, shuffle=True)
# test_ds = prepare(test_ds, shuffle=True)

In [ ]:
from tqdm import tqdm
import datetime

In [ ]:
today = str(datetime.date.today())
f1_modelcheckpoint_callback = ModelCheckpoint(
    today,
    monitor='loss',
    verbose=1,
    save_best_only=True,
    mode='min'
)

f1_early_stopping_callback = EarlyStopping(
    monitor='loss',
    patience=5,
    mode='min',
    restore_best_weights=True,
    verbose=1
)

In [ ]:
history = model.fit(
    train_ds,
    epochs=60,
    # validation_data=val_ds,
    callbacks=[
        f1_modelcheckpoint_callback,
        f1_early_stopping_callback,
    ]
)

Epoch 1/60
90/90 [==============================] - ETA: 0s - loss: 0.1215 - dice_coef_loss: 0.9931 - iou_score: 0.2300
Epoch 1: loss improved from inf to 0.12149, saving model to 2024-06-06
90/90 [==============================] - 141s 1s/step - loss: 0.1215 - dice_coef_loss: 0.9931 - iou_score: 0.2300
Epoch 2/60
90/90 [==============================] - ETA: 0s - loss: 0.0309 - dice_coef_loss: 0.9901 - iou_score: 0.2806
Epoch 2: loss improved from 0.12149 to 0.03089, saving model to 2024-06-06
90/90 [==============================] - 79s 879ms/step - loss: 0.0309 - dice_coef_loss: 0.9901 - iou_score: 0.2806
Epoch 3/60
90/90 [==============================] - ETA: 0s - loss: 0.0226 - dice_coef_loss: 0.9077 - iou_score: 0.4272
Epoch 3: loss improved from 0.03089 to 0.02259, saving model to 2024-06-06
90/90 [==============================] - 79s 872ms/step - loss: 0.0226 - dice_coef_loss: 0.9077 - iou_score: 0.4272
Epoch 4/60
90/90 [==============================] - ETA: 0s - loss: 0.012

In [ ]:
# plot the history of training
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

KeyError: 'accuracy'

## Visualize the prediction

In [ ]:
# function to match the segmentation back to the origin image
def match(img, pred_img):
  match_img = pred_img*img
  return match_img

In [ ]:
# function to average color
def average(img, i, j, r):
  sum = 0
  for x in range(i - r, i + r + 1):
    for y in range(j -r, j + r + 1):
      sum += img[x][y]
  return sum/((2*r + 1)**2)

# function to match the segmentation back to the origin image, but we just get a square area
def square_match(img, pred_img, target):
  # get top, right, bottom, left boundary

  top_flat = 0
  top_index = 512
  for i in range(0, 510):
    for j in range(0, 510):
      if average(pred_img, i, j, 2) > 0.7:
        top_flat = 1
        top_index = max(0, i - 10)
        break
    if top_flat == 1:
      break

  right_flat = 0
  right_index = 0
  for j in reversed(range(0, 510)):
    for i in range(0, 510):
      if average(pred_img, i, j, 2) > 0.7:
        right_flat = 1
        right_index = min(j + 10, 512)
        break
    if right_flat == 1:
      break

  bottom_flat = 0
  bottom_index = 0
  for i in reversed(range(0, 510)):
    for j in range(0, 510):
      if average(pred_img, i, j, 2) > 0.7:
        bottom_flat = 1
        bottom_index = min(512, i + 10)
        break
    if bottom_flat == 1:
      break

  left_flat = 0
  left_index = 512
  for j in range(0, 510):
    for i in range(0, 510):
      if average(pred_img, i, j, 2) > 0.7:
        left_flat = 1
        left_index = max(0, j - 10)
        break
    if left_flat == 1:
      break

  if target == "full":
    for i in range(512):
      for j in range(512):
        if i < top_index or i > bottom_index or j < left_index or j > right_index:
          img[i][j] = 0

  if target == "crop":
    if left_index == 512 and top_index == 512 and right_index == 0 and bottom_index == 0:
      print('512x512')
      return 0
    else:
      return img[top_index:bottom_index, left_index:right_index]

  return img

In [ ]:
# X is already normalized
def predict(X, model):
  pred_y = model.predict(X)
  pred_y[pred_y > 0.5] = 1
  pred_y[pred_y != 1] = 0
  return pred_y

In [ ]:
def predict_one_image(img, model):
  resized_img = cv2.resize(img, (512, 512))
  X = np.reshape(resized_img, (1, resized_img.shape[0], resized_img.shape[1], 1))
  normalized_X = X/255
  normalized_X = np.rollaxis(normalized_X, 3, 1)
  pred_y = model.predict(normalized_X)
  pred_y[pred_y > 0.5] = 1
  pred_y[pred_y != 1] = 0
  pred_img = np.reshape(pred_y[0]*255, (512, 512))
  match_img = square_match(resized_img, np.reshape(pred_y[0], (512, 512)), "crop")

  return pred_img, match_img

In [ ]:
# function to show the predictions (input: 690x690 image as array)
def show_predictions(img, label):
  resized_img = cv2.resize(img, (512, 512))
  pred_img1, match_img1 = predict_one_image(img, model)

  # plot the origin and predictions for
  fig, ax = plt.subplots(2, 2, figsize=(30, 10))
  ax[0][0].imshow(resized_img, cmap='gray', interpolation='nearest')
  ax[0][1].imshow(match_img1, cmap='gray', interpolation='nearest')
  ax[1][0].imshow(pred_img1, cmap='gray', interpolation='nearest')
  ax[1][1].imshow(label, cmap='gray', interpolation='nearest')
  plt.show()

In [ ]:
path = os.path.join('data2/X/AN_NGUYEN_VANse000_118.jpg')
img = cv2.imread(path, 0)
path = os.path.join('data2/y/AN_NGUYEN_VANse000_118.jpg')
label = cv2.imread(path, 0)

# show_predictions(X[634]*255)
show_predictions(img, label)

error: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [ ]:
img

In [ ]:
model.save("catheter_detect.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Evaluate the performance of the models

In [ ]:
# note: X is already normalized
def recall(model, X, y):

  pred = predict(X, model)
  pred[pred > 0.5] = 1
  pred[pred != 1] = 0

  multiply = pred*y
  s = np.sum(multiply)

  n = np.sum(y)

  return s/n

In [ ]:
# note: X is already normalized
def precision(model, X, y):

  pred = predict(X, model)
  pred[pred > 0.5] = 1
  pred[pred != 1] = 0

  multiply = pred*y
  s = np.sum(multiply)

  n = np.sum(pred)

  return s/n

In [ ]:
# note: X is already normalized
def f1_score(model, X, y):
  r = recall(model, X, y)
  p = precision(model, X, y)

  return 2*r*p/(r+p)

In [ ]:
# note: X is already normalized
def pixel_accuracy(model, X, y):

  pred = predict(X, model)
  pred[pred > 0.5] = 1
  pred[pred != 1] = 0

  o = np.absolute(pred-y)
  s = np.sum(o)

  n = X.shape[0]*512*512

  return 1-s/n

In [ ]:
def fix_x(x):
  return np.rollaxis(x, 3, 1)

In [ ]:
X_test = X_val
y_test = y_val

In [ ]:
X_test = fix_x(X_test)
y_test = fix_x(y_test)

In [ ]:
X_test.shape

(179, 1, 512, 512)

In [ ]:
print('recall: ', recall(model, X_test, y_test))
print('precision: ', precision(model, X_test, y_test))
print('f1_score: ', f1_score(model, X_test, y_test))
print('pixel_accuracy ', pixel_accuracy(model, X_test, y_test))

6/6 [==============================] - 27s 5s/step
recall:  0.91510206
6/6 [==============================] - 4s 804ms/step
precision:  0.92099
6/6 [==============================] - 4s 797ms/step
f1_score:  0.918036616611208
6/6 [==============================] - 4s 798ms/step
pixel_accuracy  0.9991900268213709
